# Proyecto Parte 2

Fase de redes neuronales de la segunda parte del proyecto del curso de Minería de datos

## Importando librerías

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

## Carga de datos

Los datos deben estar cargados al espacio de google colab

In [16]:
# Carga de df_egresos.csv
data_egresos = pd.read_csv('df_egresos.csv')
data_egresos.head()

,EJERCICIO,ENTIDAD,UNIDAD_EJECUTOR,PROGRAMA,SUBPROGRAMA,PROYECTO,ACTIVIDAD,OBRA,DEPTO,MUNICIPIO,...,GRUPO_GASTO,SUBGRUPO_GASTO,RENGLON,COD_ECO,FUENTE,ORGANISMO,CORRELATIVO,ASIGNADO,VIGENTE,DEVENGADO
0,2023,11130004,0,11,0,0,1,0,3800,3800,...,0,70,73,2111400,11,0,0,31200.0,31902.0,31876.15
1,2023,11130004,0,11,0,0,1,0,3800,3800,...,0,70,74,2111400,11,0,0,3530877.0,4093125.0,4064402.98
2,2023,11130004,0,11,0,0,1,0,3800,3800,...,0,70,72,2111400,11,0,0,647810.0,642863.0,642388.17
3,2023,11130004,0,11,0,0,1,0,3800,3800,...,0,70,71,2111400,11,0,0,647810.0,647116.0,647065.52
4,2023,11130004,0,11,0,0,1,0,3800,3800,...,0,10,18,2111100,11,0,0,55189912.0,64037433.0,64005185.92


In [4]:
# Carga de df_ingresos.csv

data_ingresos = pd.read_csv('df_ingresos.csv')
data_ingresos.head()

,EJERCICIO,ENTIDAD,DEPTO,CLASE,SECCION,GRUPO,COD_ECONOMICO,RECURSO,ASIGNADO,VIGENTE,DEVENGADO
0,2023,11130004,0,11000,11100,11110,1131100,11110,79785000.0,79785000.0,0.000000e+00
1,2023,11130004,0,23000,23100,23110,1315110,23110,45000000.0,45000000.0,0.000000e+00
2,2023,11130004,100,11000,11100,11110,1131100,11110,0.0,0.0,1.016677e+08
3,2023,11130005,0,11000,11200,11210,1132100,11210,206145050.0,206145050.0,0.000000e+00
4,2023,11130005,0,11000,11200,11290,1132200,11290,1000000.0,1000000.0,0.000000e+00


## Implementand red neuronal para egresos

Se tratará de construir para predecir si el presupuesto asignado para egresos es mayor a 1.5 Millones de quetzales, utilizando las variables de departamento, unidad ejecutora y el programa.

In [7]:
# Primero se desordenaran los registro de data_egresos

data_egresos = data_egresos.sample(frac=1, random_state=43).reset_index(drop=True)
data_egresos.head()

,EJERCICIO,ENTIDAD,UNIDAD_EJECUTOR,PROGRAMA,SUBPROGRAMA,PROYECTO,ACTIVIDAD,OBRA,DEPTO,MUNICIPIO,...,GRUPO_GASTO,SUBGRUPO_GASTO,RENGLON,COD_ECO,FUENTE,ORGANISMO,CORRELATIVO,ASIGNADO,VIGENTE,DEVENGADO
0,2022,11130009,259,13,0,0,7,0,1800,1801,...,200,220,223,2112200,41,1204,76,0.0,0.0,0.0
1,2023,11130008,319,13,0,0,2,0,1900,1906,...,0,70,72,2111400,11,0,0,48231.0,51132.0,51131.2
2,2021,11130009,258,13,0,0,4,0,1700,1712,...,200,260,268,2112200,29,101,8,9995.0,0.0,0.0
3,2019,11130005,211,13,0,0,1,0,100,101,...,300,320,328,2211300,11,0,0,0.0,548100.0,24975.0
4,2017,11130008,316,13,0,0,2,0,1600,1612,...,400,410,416,2171100,11,0,0,47250.0,47250.0,40500.0


In [18]:
# Se seleccionana las columnas que se usaran como variables independientes

X = data_egresos[['UNIDAD_EJECUTOR','DEPTO', 'PROGRAMA']]

# Se selecciona la variable dependiente

data_egresos['MAYOR_1M'] = data_egresos['ASIGNADO'].apply(lambda x: 0 if x < 1000000 else 1)

y = data_egresos['MAYOR_1M']

# ver datos en data_egresso con mayor_1_5m igual a 1



In [19]:
# Importando librerias para redes neuronales

from sklearn.model_selection import train_test_split

# Limpiando datos en caso de nulso o cadenas vacías

X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size = 0.2, random_state = 42)

X_train = X_train.replace(' ', np.nan).dropna().astype('float32')
y_train = y_train[X_train.index].astype('float32')

X_test = X_test.replace(' ', np.nan).dropna().astype('float32')
y_test = y_test[X_test.index].astype('float32')

In [27]:
# Armando capas de red neuronal

model = Sequential()
model.add(Dense(100, input_dim = 3, activation = 'relu'))
model.add(Dense(70, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [28]:
# Compilando modelo

model.compile(loss ='binary_crossentropy', optimizer ='adam', metrics =['accuracy'])

In [29]:
# Entrenando modelo

model.fit(X_train, y_train, epochs = 50, batch_size = 128, validation_data=(X_test, y_test))

Epoch 1/50
7136/7136 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - accuracy: 0.9439 - loss: 0.2960 - val_accuracy: 0.9489 - val_loss: 0.1966
Epoch 2/50
7136/7136 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - accuracy: 0.9491 - loss: 0.1942 - val_accuracy: 0.9490 - val_loss: 0.1935
Epoch 3/50
7136/7136 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - accuracy: 0.9489 - loss: 0.1936 - val_accuracy: 0.9489 - val_loss: 0.1933
Epoch 4/50
7136/7136 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - accuracy: 0.9489 - loss: 0.1932 - val_accuracy: 0.9489 - val_loss: 0.1927
Epoch 5/50
7136/7136 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - accuracy: 0.9493 - loss: 0.1914 - val_accuracy: 0.9489 - val_loss: 0.1921
Epoch 6/50
7136/7136 ━━━━━━━━━━━━━━━━━━━━ 23s 3ms/step - accuracy: 0.9492 - loss: 0.1916 - val_accuracy: 0.9490 - val_loss: 0.1916
Epoch 7/50
7136/7136 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - accuracy: 0.9492 - loss: 0.1903 - val_accuracy: 0.9491 - val_loss: 0.1896
Epoch 8/50
7136/7136 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - accuracy: 0.9490 - loss: 0

In [30]:
# Poniendo a prueba la red neuronal

loss, acc = model.evaluate(X_test, y_test)
print(acc*100)

7136/7136 ━━━━━━━━━━━━━━━━━━━━ 9s 1ms/step - accuracy: 0.9491 - loss: 0.1780
94.89063620567322


In [33]:
# Realizando pruebas

# Caso 1
# UNIDAD_EJECUTOR = 330 : DIRECCIÓN DEPARTAMENTAL DE EDUCACIÓN QUICHÉ NORTE
# DEPARTAMENTO = 1400 : Quiché
# PROGRAMA = 13:  EDUCACIÓN ESCOLAR BÁSICA

caso1 = np.array([[330, 1400, 13]])
print(model.predict(caso1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
[[0.13236746]]


In [41]:
# Caso 2
# UNIDAD_EJECUTOR = 202 : VICEPRESIDENCIA DE LA REPÚBLICA
# DEPARTAMENTO = 100 : Guatemala
# PROGRAMA = 11:  DIRECCIÓN Y COORDINACIÓN EJECUTIVA

caso2 = np.array([[202, 100, 11]])
print(model.predict(caso2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
[[0.10436723]]


In [43]:
# Caso 3
# UNIDAD_EJECUTOR = 0 : SERVICIOS DE LA DEUDA PUBLICA
# DEPARTAMENTO = 3900 : SERVICIOS DE LA DEUDA PÚBLICA
# PROGRAMA = 99:  SERVICIOS DE LA DEUDA PÚBLICA

caso3 = np.array([[0, 3900, 99]])
print(model.predict(caso3))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
[[0.3857191]]
